Comparison of data related to all countries, and continents():
1. Colection of data from countries
2.Collection of data from continents
3. Couning manually average related to continents based on counties data
4.Comparison of diferences with WHO data


In [ ]:
import pandas as pd

whole_data = pd.read_csv(
    "./../data/source/2- burden-disease-from-each-mental-illness(1).csv"
)
# data_orgin = whole_data["Entity"].unique()
whole_data.columns = [
    col.split(" Cause: ")[1].strip() if " Cause:" in col else col
    for col in whole_data.columns
]

# WHO data collection - data inspection
WHO = whole_data[whole_data["Entity"].str.contains(r"\(WHO\)", regex=True)]
who_regions = pd.read_csv("./../data/source/who_region_mapping.csv")

Fixing names of region, and attributing ISO Code to country

In [ ]:
import pycountry

# The function returns ISO3 code for country
def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Manual correction of names of countries present in data base
manual_corrections = {
    "England": "United Kingdom",
    "Cote d'Ivoire": "Côte d’Ivoire",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "Republic of Korea": "Korea, Republic of",
    "Russia": "Russian Federation",
    "Tanzania": "Tanzania, United Republic of",
    "Vietnam": "Viet Nam",
    "Iran": "Iran, Islamic Republic of",
    "Syria": "Syrian Arab Republic",
    "United States": "United States of America",
    "Palestine": "Palestine, State of",
    "Micronesia": "Micronesia, Federated States of",
    "Moldova": "Moldova, Republic of",
    "Bolivia": "Bolivia, Plurinational State of",
    "Venezuela": "Venezuela, Bolivarian Republic of",
}

# Appling correction to dataset
countries = whole_data[~whole_data["Entity"].str.contains(r"\(", regex=True)].copy()
countries = whole_data[~whole_data["Entity"].str.contains(r"\(", regex=True)]
countries["Entity_Corrected"] = countries["Entity"].replace(manual_corrections)
countries["Code"] = countries["Entity_Corrected"].apply(get_iso3)

missing_iso = countries[countries["Code"].isna()]  # countries without ISO3 country code

who_regions["ISO3"] = who_regions["Country"].apply(get_iso3)
who_regions_small = who_regions[["ISO3", "WHO_Region"]]
countries = countries.dropna(subset=["Code"])
countries = countries.merge(
    who_regions_small, left_on="Code", right_on="ISO3", how="left"
)
countries = countries.dropna(subset=["WHO_Region"])
countries.drop(columns=["ISO3", "Code", "Entity_Corrected"], inplace=True)
countries.to_csv("./../data/who_countries")

C:\Users\oktaw\AppData\Local\Temp\ipykernel_18956\2913781542.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries["Entity_Corrected"] = countries["Entity"].replace(manual_corrections)
C:\Users\oktaw\AppData\Local\Temp\ipykernel_18956\2913781542.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries["Code"] = countries["Entity_Corrected"].apply(get_iso3)


Couning manually arithmetric average related to WHO regions based on countries data

In [ ]:
WHO = whole_data[whole_data["Entity"].str.contains(r"\(WHO\)", regex=True)]
who_regions2 = countries["WHO_Region"].unique()  # my files

# Arithmetric acerage calculations for each WHO region and year
for region in who_regions2:
    region_data = countries[countries["WHO_Region"] == region]
    region_data = region_data.drop(columns=["Entity", "WHO_Region"])
    region_data = pd.melt(
        region_data, id_vars=["Year"], var_name="Disorder", value_name="DALYs"
    )
    disorders = region_data["Disorder"].unique()
    years = region_data["Year"].unique()

    new_region = pd.DataFrame(columns=["Year", "Disorder", "DALYs"])
    for disorder in disorders:
        data = region_data[region_data["Disorder"] == disorder]
        for year in years:
            year_data = data[data["Year"] == year]
            record = year_data["DALYs"].mean()
            new_row = pd.DataFrame(
                [
                    {
                        "Year": year,
                        "Disorder": disorder,
                        "DALYs": record,
                        "Region": region,
                        "Source": "Average",
                    }
                ]
            )
            new_region = pd.concat([new_region, new_row], ignore_index=True)
    new_region.to_csv("./../data/who_comp/new_" + region)

# Separating given data by WHO regions
WHO = whole_data[whole_data["Entity"].str.contains(r"\(WHO\)", regex=True)]
regions = WHO["Entity"].unique()
print(regions)

path = "./../data/who_comp/who_"

WHO = WHO.drop(columns=["Code"])

for region in regions:
    data = WHO[WHO["Entity"] == region]
    data = data.drop(columns=["Entity"])
    data = pd.melt(data, id_vars="Year", var_name="Disorder", value_name="DALYs")

    data["Region"] = region
    data["Source"] = "WHO"

    data.to_csv(path + region)

C:\Users\oktaw\AppData\Local\Temp\ipykernel_18956\2692165608.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_region = pd.concat([new_region, new_row], ignore_index=True)
C:\Users\oktaw\AppData\Local\Temp\ipykernel_18956\2692165608.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_region = pd.concat([new_region, new_row], ignore_index=True)
C:\Users\oktaw\AppData\Local\Temp\ipykernel_18956\2692165608.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-N

['African Region (WHO)' 'Eastern Mediterranean Region (WHO)'
 'European Region (WHO)' 'Region of the Americas (WHO)'
 'South-East Asia Region (WHO)' 'Western Pacific Region (WHO)']


Whole data - one dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

path_new = "./../data/who_comp/new_*"
path_who = "./../data/who_comp/who*"
# path = "./../data/who_comp/"

files_new = glob.glob(path_new)
files_who = glob.glob(path_who)

data_list = [pd.read_csv(f) for f in files_new]
combined_df = pd.concat(data_list, ignore_index=True)
data_list_2 = [pd.read_csv(f) for f in files_who]
combined_df2 = pd.concat(data_list_2, ignore_index=True)

x = [combined_df, combined_df2]

final_data = pd.concat(x, ignore_index=True)
final_data.to_csv("./../results/final_comparison.csv", index=False)